In [ ]:
!pip install openai

In [ ]:
import json
from openai import OpenAI # GPT-4 API
from tqdm import tqdm
from google.colab import userdata
from string import Template

In [ ]:

# GPT-4 API 키 설정
client = OpenAI(api_key=userdata.get('openapi_key'))
# 데이터셋 로드
file_path = "2023_11_KICE.json"  # JSON 파일 경로

# JSON 파일 로드
def load_json(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return json.load(file)

def prediction(problem):
    """
    GPT-4가 문제에 대한 정답을 예측합니다.
    Args:
        problem (dict): JSON 형식의 문제 데이터
    Returns:
        str: GPT-4가 예측한 정답 (선택지 번호)
    """
    # 지문 및 선택지 추출
    paragraph = problem.get("paragraph", "")
    question = problem["question"]
    question_plus = problem.get('question_plus', None)
    choices = problem["choices"]

    # Zero-shot CoT 프롬프트 템플릿
    zero_shot_cot_en_prompt = Template("""

    Paragraph:
    ${paragraph}

    Question:
    ${question}

    More info:
    ${question_plus}

    Choices:
    1 - ${choice1}
    2 - ${choice2}
    3 - ${choice3}
    4 - ${choice4}
    5 - ${choice5}

    Choice one in 5 choices.
    Your answer should be a single number (1, 2, 3, 4, 5) and nothing else.
    Let's think step by step.
    """)

    # 프롬프트 구성
    prompt = zero_shot_cot_en_prompt.substitute(
        paragraph=paragraph,
        question=question,
        question_plus=question_plus,
        choice1=choices[0],
        choice2=choices[1],
        choice3=choices[2],
        choice4=choices[3],
        choice5=choices[4],
    )

    # GPT-4 API 호출
    try:
        response = client.chat.completions.create(
            model="gpt-4o",  # GPT-4 모델 선택
            messages=[
                {"role": "system", "content": "As a smart student answer the given question. Read the paragraph, and select only one answer between 5 choices."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=4096,  # 충분한 길이 허용
            temperature=0.5,
            n=1
        )
        # 정답 추출
        answer = response.choices[0].message.content
        # 답변에서 숫자만 추출
        for option in ['1', '2', '3', '4', '5']:
            if option in answer:
                return option

        return None  # 유효한 선택지가 없을 경우
    except Exception as e:
        print(f"Error during GPT-4 prediction: {e}")
        return None

# 모든 문제에 대해 예측 수행
def predict_all(data):
    predictions = []

    for item in tqdm(data):
        paragraph = item.get("paragraph", "")
        problems = item["problems"]

        for problem in problems:
            problem_with_paragraph = {**problem, "paragraph": paragraph}
            predicted_answer = prediction(problem_with_paragraph)
            predictions.append({
                "predicted_answer": predicted_answer,
                "true_answer": str(problem["answer"]),
                "score": problem["score"]
            })

    return predictions

# 점수 계산 함수
def calculate_score(predictions):
    total_score = 0
    correct_answers = 0
    total_questions = len(predictions)

    for result in predictions:
        if result["predicted_answer"] == result["true_answer"]:
            total_score += result["score"]
            correct_answers += 1

    accuracy = correct_answers / total_questions * 100 if total_questions > 0 else 0

    return {
        "total_score": total_score,
        "correct_answers": correct_answers,
        "total_questions": total_questions,
        "accuracy": accuracy
    }



In [ ]:
data = load_json(file_path)


# 예측 수행
predictions = predict_all(data)


# 점수 계산
results = calculate_score(predictions)

# 결과 출력
print(f"Total Score: {results['total_score']}")
print(f"Correct Answers: {results['correct_answers']}")
print(f"Total Questions: {results['total_questions']}")
print(f"Accuracy: {results['accuracy']:.2f}%")

if results['total_score'] >= 80:
    print("과제 통과!")
else:
    print("과제 실패")

100%|██████████| 11/11 [01:18<00:00,  7.16s/it]

Total Score: 87
Correct Answers: 39
Total Questions: 45
Accuracy: 86.67%
과제 통과!
